In [1]:
from downloader import download
from calendar import monthrange
from datetime import datetime
import json

In [2]:
ely_codes = ['01', '02', '03', '04', '08', '09', '10', '12', '14']
with open('../elyjenmappaus/ely_lamid_mapping.json', 'r') as mappings:
    lam_id_mapping = json.loads(mappings.read())

In [3]:
def create_file_name(lam_id, year, day_number):
    year_short = year % 100
    return f"lamraw_{lam_id}_{year_short}_{day_number}.csv"

def first_and_last_day_number(month, year):
    first_day = datetime(year, month, 1)
    last_day = datetime(year, month, monthrange(year, month)[1])
    
    first = first_day.timetuple().tm_yday
    last = last_day.timetuple().tm_yday
    
    return (first, last)

def create_download_parameters_for_months(lam_id, year, months, ely_code, path):
    parameters = []
    for month in months:
        first_day, last_day = first_and_last_day_number(month, year)

        for day_number in range(first_day, last_day + 1):
            parameters.append((year, ely_code, create_file_name(lam_id, year, day_number), path))    
    
    
    return parameters

In [4]:
parameters = []
selected_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
selected_year = 2017

# create params for each ELY code and it's LAM
for ely in ely_codes:
    lams = lam_id_mapping[ely.strip("0")]
    for lam in lams:
        parameters = parameters + create_download_parameters_for_months(lam, selected_year, selected_months, ely)

In [5]:
external_drive_path = "YOUR_PATH\\data\\raw_data"
failed_and_existed = download(parameters)

In [6]:
len(failed_and_existed[0])

5641

In [20]:
int('4') in lam_id_mapping['1']

True

In [4]:
def find_ely(lam_id):
    for key in lam_id_mapping.keys():
        if int(lam_id) in lam_id_mapping[key]:
            return key
    return -1

def append_to_key_value(dictionary, key, value):
    if key in dictionary:
        dictionary[key].append(value)
    else:
        dictionary[key] = [value]
        
def add_missing_dict(dictionary, key):
    if key not in dictionary:
        dictionary[key] = {}

def create_errored_json(errored):
    errored_dict = {}
    for msg in errored:
        filename = msg.split(" ")[-1]
        lam_id = filename.split("_")[1]
        day = filename.split("_")[-1].split(".")[0]
        ely = find_ely(lam_id)
        add_missing_dict(errored_dict, ely)
        append_to_key_value(errored_dict[ely], lam_id, day)
        
    return errored_dict

In [29]:
errored_json = create_errored_json(failed_and_existed[0])

In [5]:
import json

In [6]:
def save_json(json_data, filename):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, sort_keys=True)


In [42]:
save_json(errored_json, 'errored_file_downloads_2017_1-12.json')

In [7]:
parameters = []
selected_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
selected_year = 2017
external_drive_path = "YOUR_PATH\\data\\raw_data"

# create params for each ELY code and it's LAM
for ely in ely_codes:
    lams = lam_id_mapping[ely.strip("0")]
    for lam in lams:
        parameters = parameters + create_download_parameters_for_months(lam, selected_year, selected_months, ely, external_drive_path)

In [8]:
failed_and_existed = download(parameters)
print(f"Failed count: {len(failed_and_existed[0])}")
errored_json = create_errored_json(failed_and_existed[0])
save_json(errored_json, f"{external_drive_path}\\errored_file_downloads_2017_1-12.json")

In [46]:
len(failed_and_existed)

2

In [47]:
errored_json = create_errored_json(failed_and_existed[0])

In [48]:
save_json(errored_json, 'errored_file_downloads_2016_1-12.json')

In [54]:
error_len = 0
for key in errored_json.keys():
    for lam_key in errored_json[key]:
        error_len += len(errored_json[key][lam_key])
        
print(f"{error_len} file downloads failed")

59803 file downloads failed
